https://blog.roboflow.com/how-to-use-yolov8-with-sam/

# Setting Up

In [14]:
import ultralytics
from IPython.display import display, Image
import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt

# Set the device for GPU acceleration
device = "cuda"

# Check Ultralytics version and setup completion
ultralytics.checks()

# Set the first_run flag to False after the initial run
first_run = True

if first_run:
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

Ultralytics YOLOv8.0.200  Python-3.9.18 torch-2.0.0 CUDA:0 (NVIDIA RTX A4500, 20470MiB)
Setup complete  (128 CPUs, 255.9 GB RAM, 252.0/953.3 GB disk)
ERROR: Invalid requirement: "'git+https://github.com/facebookresearch/segment-anything.git'"
'wget' is not recognized as an internal or external command,
operable program or batch file.


# Loading the Dataset

In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("vkr-v2").project("vkrrr")
dataset = project.version(5).download("yolov8")

ModuleNotFoundError: No module named 'roboflow'

# Running YOLOv8 Inference

In [7]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

path = r'C:\Users\WANGH0M\gaussian-splatting\data_bonsai_new\images'
name = path + r'\IMG_6056.png'
# Perform object detection on the image
results = model.predict(source=name, conf=0.25)


image 1/1 C:\Users\WANGH0M\gaussian-splatting\data_bonsai_new\images\IMG_6056.png: 640x480 1 potted plant, 1 dining table, 1 vase, 89.3ms
Speed: 5.0ms preprocess, 89.3ms inference, 24.0ms postprocess per image at shape (1, 3, 640, 480)


# Extracting the Bounding Box

In [8]:
for result in results:
    boxes = result.boxes
bbox = boxes.xyxy.tolist()[0]
print(bbox)

[587.7847900390625, 1084.9083251953125, 1065.2694091796875, 1613.078857421875]


# Convert Bounding Box to Segmentation Mask using SAM

In [11]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

image = cv2.cvtColor(cv2.imread(name), cv2.COLOR_BGR2RGB)
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)
predictor.set_image(image)

ModuleNotFoundError: No module named 'segment_anything'

In [12]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))    

## Mask

In [13]:
input_box = np.array(bbox)
masks, _, _ = predictor.predict(
    point_coords=None,
    point_labels=None,
    box=bbox[None, :],
    multimask_output=False,
)

NameError: name 'predictor' is not defined

## Plot mask + Bbox

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
show_mask(masks[0], plt.gca())
show_box(input_box, plt.gca())
plt.axis('off')
plt.show()

# Background Removal

In [ ]:
segmentation_mask = masks[0]

# Convert the segmentation mask to a binary mask
binary_mask = np.where(segmentation_mask > 0.5, 1, 0)
white_background = np.ones_like(image) * 255

# Apply the binary mask
new_image = white_background * (1 - binary_mask[..., np.newaxis]) + image * binary_mask[..., np.newaxis]

plt.imshow(new_image.astype(np.uint8))
plt.axis('off')
plt.show()